## 1. データをクローリングする

In [114]:
! mkdir -p html excel csv json tokenized vector log

In [15]:
# 事故事例(Excel系)と、事例・状況・原因・対策（HTML系）の2種類を取得する。
! ./get_doc.sh > ./log/get_doc.log 2>&1 

## 2. 取得したデータをCSVまたはJSONに変換する。
* 事故事例のExcelをCSVに変換する。
* 事例・状況・原因・対策のHTMLはJSONに変換する。

In [28]:
# Excel to CSV
!python excel_to_csv.py ./excel/ ./csv/ > ./log/excel_to_csv.log 2>&1

In [30]:
# HTML to JSON
!python html_to_json.py ./html/ ./json/ > ./log/html_to_json.log 2>&1 

## 3. ElasticsearchにIndexを登録する

In [62]:
# 事例・状況・原因・対策用のインデックスを登録
! curl -XPUT -H "Content-Type: application/json" elasticsearch:9200/anzen -d @es_anzen_schema.txt

{"acknowledged":true,"shards_acknowledged":true,"index":"anzen"}

In [95]:
# 事例用のインデックスを登録
! curl -XPUT -H "Content-Type: application/json" elasticsearch:9200/accident -d @es_accident_schema.txt

{"acknowledged":true,"shards_acknowledged":true,"index":"accident"}

## 4. Elasticsearchにデータをロードする

In [100]:
# 事例・状況・原因・対策データをロード(anzenインデックス)
! ./load_anzen_bulk_es.sh > ./log/load_anzen_bulk_es.log 2>&1

In [96]:
# 事例用データをロード(accidentインデックス)
!python load_accident_es.py --host elasticsearch --index accident --input_dir ./csv/  > ./log/load_accident_es.log 2>&1

## 5. Elasticsearchでトークナイズし、結果をCSV形式とテキスト形式で保存する

In [115]:
# 事例・状況・原因・対策データの文章をトークナイズし、CSVとTXTファイルとして保存する
!python es_anzen_tokenize.py --host elasticsearch --index anzen --output tokenized/anzen_tokenized

In [116]:
# 事例用データの文章をトークナイズし、CSVとTXTファイルとして保存する
!python es_accident_tokenize.py --host elasticsearch --index accident --output tokenized/accident_tokenized

## 6. GloVeで単語を学習する

In [109]:
# GloVeを取得する。
! git clone https://github.com/stanfordnlp/GloVe.git
! cd GloVe && make

fatal: destination path 'GloVe' already exists and is not an empty directory.
mkdir -p build
gcc -c src/vocab_count.c -o build/vocab_count.o -lm -pthread -O3 -march=native -funroll-loops -Wall -Wextra -Wpedantic
gcc -c src/cooccur.c -o build/cooccur.o -lm -pthread -O3 -march=native -funroll-loops -Wall -Wextra -Wpedantic
src/cooccur.c: In function ‘merge_files’:
src/cooccur.c:180:9: warning: ignoring return value of ‘fread’, declared with attribute warn_unused_result [-Wunused-result]
         fread(&new, sizeof(CREC), 1, fid[i]);
         ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
src/cooccur.c:190:5: warning: ignoring return value of ‘fread’, declared with attribute warn_unused_result [-Wunused-result]
     fread(&new, sizeof(CREC), 1, fid[i]);
     ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
src/cooccur.c:203:9: warning: ignoring return value of ‘fread’, declared with attribute warn_unused_result [-Wunused-result]
         fread(&new, sizeof(CREC), 1, fid[i]);
         ^~~~~~~~~~~~~~~~~~~~~~~~~~

In [117]:
# トークナイズしたanzenとaccidentのテキストファイルを結合する。
! cat tokenized/anzen_tokenized.txt tokenized/accident_tokenized.txt > tokenized/all_tokens.txt

In [119]:
# GloVeで学習する。
! ./glove.sh 


$ GloVe/build/vocab_count -min-count 0 -verbose 2 < tokenized/all_tokens.txt > ./vector/glove_vocab.txt
BUILDING VOCABULARY
Processed 0 tokens.100000 tokens.200000 tokens.300000 tokens.400000 tokens.500000 tokens.600000 tokens.700000 tokens.800000 tokens.900000 tokens.1000000 tokens.1100000 tokens.1200000 tokens.1300000 tokens.1400000 tokens.1500000 tokens.1600000 tokens.1700000 tokens.1800000 tokens.1900000 tokens.2000000 tokens.2100000 tokens.2200000 tokens.2300000 tokens.2400000 tokens.2500000 tokens.2600000 tokens.2700000 tokens.2800000 tokens.2900000 tokens.3000000 tokens.3100000 tokens.3200000 tokens.Processed 3208826 tokens.
Counted 27931 unique words.
Using vocabulary of size 27931.

$ GloVe/build/cooccur -memory 4.0 -vocab-file ./vector/glove_vocab.txt -verbose 2 -window-size 15 < tokenized/all_tokens.txt > ./vector/glove_cooccurrence.bin
COUNTING COOCCURRENCES
window size: 15
context: symmetric
max product: 13752509
overflow length: 38028356
Reading vocab from file "./vector

# SCDVで文章をベクトル化する

In [149]:
# anzenとaccidentのCSVをマージする。ベクトル化に必要な列のみ選別し、列名を統一する。
!python merge_csv.py --input_anzen tokenized/anzen_tokenized.csv --input_accident tokenized/accident_tokenized.csv --output_csv tokenized/merge_tokenized.csv

In [152]:
!python scdv.py --csv_file ./tokenized/merge_tokenized.csv 

sys:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
Clustering Done...
Cluster Assignments Saved...
Probabilities of Cluster Assignments saved...
100%|██████████████████████████████████| 13155/13155 [00:00<00:00, 17787.37it/s]
train size:118394  vector size:1000
test size:13155  vector size:1000
Test start...
Report
              precision    recall  f1-score   support

       cause   0.557692  0.435918  0.489343      1264
    measures   0.695862  0.706088  0.700938      1429
   situation   0.629565  0.500000  0.557352      1448
       title   0.468750  0.297619  0.364078       252
      その他の事業   0.581882  0.341513  0.430412       489
      保健・衛生業   0.039216  0.400000  0.071429         5
        保健衛生   0.043478  0.500000  0.080000         2
       保健衛生業   0.181818  0.347826  0.238806        46
          商業   0.630972  0.474597  0.541726       807
         官公署   0.023256  0.333333  0.043478         3
         建設業   0.692008  0.81063

In [155]:
# 全データについてSCDVベクトルを算出し、Elasticsearchに登録する
!python scdv_to_es.py --host elasticsearch --input_csv ./tokenized/merge_tokenized.csv > ./log/scdv_to_es.log 2>&1 